To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton

!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Unsloth

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.2.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [3]:
from datasets import load_dataset

ds = load_dataset("maharnab/hindi_instruct")


In [4]:
from datasets import Dataset

ds2 = ds["train"]

def process_example(conv):
    return {"conversations": [
        {"content": conv["input"], "role": "user"},
        {"content": conv["output"], "role": "assistant"},
    ]}

ds2 = ds2.map(process_example, remove_columns=["input", "output", "instruction"], num_proc=4)  # Adjust num_proc based on your CPU

print(ds2)


Map (num_proc=4):   0%|          | 0/7644 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations'],
    num_rows: 7644
})


In [6]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = ds2

We now use `standardize_sharegpt` to convert ShareGPT style datasets into HuggingFace's generic format. This changes the dataset from looking like:
```
{"from": "system", "value": "You are an assistant"}
{"from": "human", "value": "What is 2+2?"}
{"from": "gpt", "value": "It's 4."}
```
to
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [7]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/7644 [00:00<?, ? examples/s]

Map:   0%|          | 0/7644 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [8]:
dataset[5]["conversations"]

[{'content': ' क्या चित्र को काव्य में भी इस्तेमाल किया जा सकता है? ',
  'role': 'user'},
 {'content': "हां, चित्र को काव्य में भी इस्तेमाल किया जा सकता है। चित्र काव्य के एक महत्वपूर्ण भाग के रूप में उपयोग होता है, जिसे 'काव्य कला' कहा जाता है। इससे काव्य की सुंदरता, भावनाओं का आवेश और पाठक की ध्यान केंद्रित की जा सकती है।",
  'role': 'assistant'}]

And we see how the chat template transformed these conversations.

**[Notice]** Llama 3.1 Instruct's default chat template default adds `"Cutting Knowledge Date: December 2023\nToday Date: 26 July 2024"`, so do not be alarmed!

In [9]:
dataset[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n क्या चित्र को काव्य में भी इस्तेमाल किया जा सकता है? <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nहां, चित्र को काव्य में भी इस्तेमाल किया जा सकता है। चित्र काव्य के एक महत्वपूर्ण भाग के रूप में उपयोग होता है, जिसे 'काव्य कला' कहा जाता है। इससे काव्य की सुंदरता, भावनाओं का आवेश और पाठक की ध्यान केंद्रित की जा सकती है।<|eot_id|>"

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/7644 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [11]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/7644 [00:00<?, ? examples/s]

We verify masking is actually done:

In [12]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n क्या चित्र को काव्य में भी इस्तेमाल किया जा सकता है? <|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nहां, चित्र को काव्य में भी इस्तेमाल किया जा सकता है। चित्र काव्य के एक महत्वपूर्ण भाग के रूप में उपयोग होता है, जिसे 'काव्य कला' कहा जाता है। इससे काव्य की सुंदरता, भावनाओं का आवेश और पाठक की ध्यान केंद्रित की जा सकती है।<|eot_id|>"

In [13]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                                               \n\nहां, चित्र को काव्य में भी इस्तेमाल किया जा सकता है। चित्र काव्य के एक महत्वपूर्ण भाग के रूप में उपयोग होता है, जिसे 'काव्य कला' कहा जाता है। इससे काव्य की सुंदरता, भावनाओं का आवेश और पाठक की ध्यान केंद्रित की जा सकती है।<|eot_id|>"

We can see the System and Instruction prompts are successfully masked!

In [14]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
2.303 GB of memory reserved.


In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,644 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.200700
2,0.985800
3,1.073600
4,0.949000
5,1.245900
6,1.360300
7,1.063800
8,1.079800
9,1.112300
10,0.991300


In [16]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

349.8113 seconds used for training.
5.83 minutes used for training.
Peak reserved memory = 4.012 GB.
Peak reserved memory for training = 1.709 GB.
Peak reserved memory % of max memory = 27.217 %.
Peak reserved memory for training % of max memory = 11.594 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [17]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Continue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContinue the fibonnaci sequence: 1, 1, 2, 3, 5, 8,<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe next two numbers in the Fibonacci sequence would be 13, 21, and so on.<|eot_id|>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [18]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "क्या हाल है?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

मैं लैबेल हो सकता हूँ और डिस्प्ले कुछ लेकिन बिना टॉकिंग करने के और भी अधिक सार्वभौमिक होता हूँ।<|eot_id|>


In [21]:
messages = [
    {"role": "user", "content": "क्या हाल है?"},
    {"role": "user", "content": "आपका नाम क्या है?"},
    {"role": "user", "content": "आप कहाँ रहते हैं?"},
    {"role": "user", "content": "आज का मौसम कैसा है?"},
    {"role": "user", "content": "आप क्या कर रहे हैं?"},
    {"role": "user", "content": "आपका पसंदीदा खाना क्या है?"},
    {"role": "user", "content": "आपका शौक क्या है?"},
    {"role": "user", "content": "आपका दिन कैसा रहा?"},
    {"role": "user", "content": "आपकी पसंदीदा किताब कौन सी है?"},
]
for message in messages:
  inputs = tokenizer.apply_chat_template(
      [message],
      tokenize = True,
      add_generation_prompt = True, # Must add for generation
      return_tensors = "pt",
  ).to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer, skip_prompt = True)
  _ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                    use_cache = True, temperature = 1.5, min_p = 0.1)


कोई पृष्ठ नहीं है कि क्या हाल है, क्योंकि मेरे पास कोई जानकारी नहीं है कि आप की पृष्ठ के बारे में सोच रहे होंगे। क्या कोई और चीज़ है?<|eot_id|>
मेरा नाम लुइस है।<|eot_id|>
मुझे कहाँ रहते हैं, यह जानने के लिए, कृपया मुझे अपने स्थान का पता दें, ताकि मैं आपको एक सटीक पता दे सकूँ।<|eot_id|>
आज का मौसम आपके क्षेत्र के अनुसार अलग-अलग होगा, लेकिन यहाँ कुछ सामान्य विचार हैं:
चाय के खेत में सुबह का सूर्य नमकीन, शांत मौसम है क्योंकि बादल नहीं हैं और नींद के बिना रात होती है, खासकर जब तक बारिश नहीं आती। ठंडा सुकून और हल्की स्प्रिंग ब्रीज़ भ
मैं विकिपीडिया पर एक बार जा रहा हूँ।<|eot_id|>
 मैं एक नाकबंद दफ्तरवाला हूँ, तो मुझे चाय से भी दुनिया नहीं समझाई देनी। मेरा पसंदीदा खाना है वेजिटेबल स्पेस्टेकी।<|eot_id|>
मैं बोर्ड खेलना और मास्टरी स्ट्रैटेजी जीतने के लिए उत्साहित हूँ!<|eot_id|>
मुझे एक बहुत अच्छा दिन मिला, और मुझे बहुत काम करने को मिला।<|eot_id|>
 मुझे बहुत सारी किताबें पसंद हैं, लेकिन मेरी पसंदीदा किताब है "गुड्स और एब्रजेशन" और "द फाइव पॉइंट्स: एक शारीरिक और मानसिक हीथ मेडिटेशन मंत्र"<|eot_

In [23]:
messages = [
    {"role": "user", "content": "एक आदमी एक पहाड़ के नीचे से चढ़ना शुरू करता है और शाम तक चोटी पर पहुँच जाता है। अगले दिन, वह ठीक उसी रास्ते से नीचे आता है। क्या यह संभव है कि किसी एक समय पर वह दोनों दिनों में ठीक उसी स्थान पर एक ही गति से हो? क्यों?"},

    {"role": "user", "content": "तीन दोस्त एक रेस्तरां में खाना खाने गए। बिल ₹750 आया। उन्होंने ₹250-₹250 करके पैसे दिए। वेटर ने ₹50 ज्यादा ले लिए थे, तो उसने ₹50 वापस किया, जिसमें से ₹30 दोस्तों ने रख लिए और ₹20 खुद रख लिया। अब अगर हर दोस्त ने ₹250-₹10 = ₹240 दिए, तो कुल ₹240 × 3 = ₹720 हुए, और वेटर ने ₹20 रखे, जिससे कुल ₹740 ही बनते हैं। बाकी ₹10 कहाँ गए?"},

    {"role": "user", "content": "एक किसान के पास एक भेड़, एक बकरी और एक बाघ है। उसे नदी पार करनी है लेकिन नाव में केवल एक चीज़ ले जा सकता है। यदि वह भेड़ और बकरी को अकेला छोड़ता है, तो बकरी भेड़ को खा जाएगी। यदि वह बकरी और बाघ को अकेला छोड़ता है, तो बाघ बकरी को खा जाएगा। वह सभी को सुरक्षित कैसे पार कर सकता है?"},

    {"role": "user", "content": "अगर 5 मशीनें 5 मिनट में 5 वस्तुएं बनाती हैं, तो 100 मशीनें 100 वस्तुएं बनाने में कितना समय लेंगी?"},

    {"role": "user", "content": "आपके पास दो रस्सियाँ हैं जो जलने में 1 घंटे का समय लेती हैं, लेकिन उनकी जलने की गति एकसमान नहीं है। आप केवल माचिस का उपयोग करके 45 मिनट कैसे माप सकते हैं?"},

    {"role": "user", "content": "एक कमरे में तीन लाइट बल्ब हैं, और बाहर तीन स्विच। आप केवल एक बार कमरे में जा सकते हैं और देख सकते हैं कि कौन सा बल्ब जल रहा है। आप कैसे पता लगाएंगे कि कौन सा स्विच किस बल्ब से जुड़ा है?"},

    {"role": "user", "content": "एक गाँव में सिर्फ दो नाई (barber) हैं। एक की दाढ़ी बहुत अच्छी तरह से कटी हुई है, और दूसरे की दाढ़ी बहुत खराब दिखती है। अगर आपको बाल कटवाने हैं, तो आप किस नाई के पास जाएंगे और क्यों?"},

    {"role": "user", "content": "एक ट्रेन A से B तक 60 km/h की गति से जाती है और B से A वापस 40 km/h की गति से आती है। पूरी यात्रा के लिए औसत गति क्या होगी?"}
]
for message in messages:
  inputs = tokenizer.apply_chat_template(
      [message],
      tokenize = True,
      add_generation_prompt = True, # Must add for generation
      return_tensors = "pt",
  ).to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(tokenizer, skip_prompt = True)
  _ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 1024,
                    use_cache = True, temperature = 1.5, min_p = 0.1)
  print("-----------")


नहीं, यह संभव नहीं है। इस दिशा में, एक इंसान कभी सूरज से अंधेरे की ओर नहीं चलता है, और न ही वह चढ़ने से नीचे की ओर। यहाँ केवल एक चीज़ होती है - उसमें जितनी अधिक गति से उतरा जाएगा, उसी दिशा से उतरा जाएगा। इसलिए, उसे दो बार दोनों दिशाओं की सामान्य गति में जाना होगा।<|eot_id|>
-----------
शुरू में 2 ₹250 और 2 ₹250 का उपयोग किया, जिसका संबंधित मात्रा $3 = $750 है। वेटर ने $50 अतिरिक्त लिया और $50 का वापसी भुगतान किया, जिसके परिणामस्वरूप कुल $250 + $250 = $500 बने। अब, अगर हर व्यक्ति $250 - $10 = $240 दिया और कुल $240 * 3 = $720 का उपयोग किया, तो दोस्तों ने $720 - $50 (वेटर का वापसी) = $670 दिया। अंत में, वेटर ने $20 रखा, जिससे कुल $750 - $20 + $670 + $10 = $730 = $750 बनता। 

इस तरह, वेटर ने अतिरिक्त $30 + $10 = $40 का भुगतान किया, जो 5 वर्ष के दौरान शेष रकम हैं। 

मान लें, $20 + $10 + $10 = $40 है। $10 भुगतान हुए। $10 + $20 + $30 = $60 की बची गई मात्रा, जिसका वेटर $10 = $50 का उपयोग करना चाह सकता है। तो, शेष $10 खुद वेटर को बनाना चाह रहा था। 

जब साथी और वेटर मिलकर $10 और $20 ले लेते, तो 

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The Eiffel Tower, located in the heart of Paris, stands tall among the city's historic and cultural landmarks. This iron structure, standing at an impressive 324 meters high, offers breathtaking views of the City of Light's iconic landscape. The Eiffel Tower was built for the 1889 World's Fair and has since become a symbol of French engineering and culture.<|eot_id|>


You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
